# Azure Cognitive Search Vector Search Code Sample with Cognitive Services Florence Vision API for Images
This code demonstrates how to use Azure Cognitive Search with Cognitive Services Florence Vision API and Azure Python SDK
## Prerequisites
To run the code, install the following packages. Please note that the `pip install azure-search-documents==11.4.0a20230509004` is currently using the Dev Feed. For instructions on how to connect to the dev feed, please visit [Azure-Python-SDK Azure Search Documents Dev Feed](https://dev.azure.com/azure-sdk/public/_artifacts/feed/azure-sdk-for-python/connect/pip). Note, that the Vision API for Vectorization of Images and Text is still in progress. We will update this sample to leverage the Vision Python SDK when it's available.

Alternatively, feel free to use the python wheel located in the build folder. 

In [ ]:
# ! pip install ../build/azure_search_documents-11.4.0b4-py3-none-any.whl  
! pip install azure-search-documents==11.4.0a20230509004  
! pip install azure-storage-blob  
! pip install requests  
! pip install python-dotenv

## Import required libraries and environment variables

In [132]:
# 1. Import libraries  
import os  
import json  
import requests  
from dotenv import load_dotenv  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,
    SearchFieldDataType,  
    SimpleField,
    FieldMapping,
    SearchableField,  
    SearchIndex,  
    VectorSearch,  
    VectorSearchAlgorithmConfiguration,  
    SearchIndexerDataContainer,  
    SearchIndexer,  
    SearchIndexerDataSourceConnection,  
    InputFieldMappingEntry,  
    OutputFieldMappingEntry,  
    SearchIndexerSkillset,
    CorsOptions,
    IndexingParameters,
    IndexerStatus,
    SearchIndexerDataContainer, SearchIndex, SearchIndexer, SimpleField, SearchFieldDataType,
    EntityRecognitionSkill, InputFieldMappingEntry, OutputFieldMappingEntry, SearchIndexerSkillset,
    CorsOptions, IndexingSchedule, SearchableField, IndexingParameters, SearchIndexerDataSourceConnection
)  
from azure.search.documents.indexes.models import WebApiSkill  
from azure.storage.blob import BlobServiceClient  
from azure.search.documents.indexes import SearchIndexerClient  
from azure.search.documents.indexes.models import (  
    SearchIndexerDataContainer,  
    SearchIndexerDataSourceConnection,  
)  

  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")  
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")  
cogSvcsEndpoint = os.getenv("COGNITIVE_SERVICES_ENDPOINT")  
cogSvcsApiKey = os.getenv("COGNITIVE_SERVICES_API_KEY")  
blob_connection_string = os.getenv("BLOB_CONNECTION_STRING")  
container_name = os.getenv("BLOB_CONTAINER_NAME")

# Connect to Blob Storage

In [133]:
# Connect to Blob Storage   
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)  
container_client = blob_service_client.get_container_client(container_name)  
blobs = container_client.list_blobs()  
first_blob = next(blobs)  
print(f"First blob in container: {first_blob.name}") 

First blob in container: 1012.png


# Create a Data Source

In [134]:
def create_data_source():  
    ds_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))  
    container = SearchIndexerDataContainer(name=container_name)  
    data_source_connection = SearchIndexerDataSourceConnection(  
        name="my-image-blob-storage",  
        type="azureblob",  
        connection_string=blob_connection_string,  
        container=container  
    )  
    data_source = ds_client.create_or_update_data_source_connection(data_source_connection)  
    return data_source  
  
data_source = create_data_source()  
print(f"Data source '{data_source.name}' created or updated") 

Data source 'my-image-blob-storage' created or updated


# Create Search Index

In [138]:
def create_index():  
    fields = [  
        SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True),  
        SimpleField(name="imageUrl", type=SearchFieldDataType.String, retrievable=True),  
        SearchField(name="title", type=SearchFieldDataType.String, searchable=True, retrievable=True),  
        SearchField(  
            name="imageVector",  
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),  
            searchable=True,  
            dimensions=1024,  
            vector_search_configuration="my-vector-config",  
        ),  
    ]  
    cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)  
  
    vector_search = VectorSearch(  
        algorithm_configurations=[  
            VectorSearchAlgorithmConfiguration(  
                name="my-vector-config",  
                kind="hnsw",  
                hnsw_parameters={  
                    "m": 4,  
                    "efConstruction": 400,  
                    "efSearch": 500,  
                    "metric": "cosine",  
                },  
            )  
        ],  
    )  
  
    index = SearchIndex(  
        name=index_name,  
        fields=fields,  
        cors_options=cors_options,  
    )  
    index_client = SearchIndexClient(service_endpoint, AzureKeyCredential(key))  
    result = index_client.create_or_update_index(index)
    return result  


# Create a Skillset

In [139]:
def create_skillset():  
    skillset_name = "my-image-skillset"  
    skill_uri = "https://image-embeddings-function-app.azurewebsites.net/api/GetImageEmbeddings" 
    skill = WebApiSkill(  
        uri=skill_uri,  
        inputs=[InputFieldMappingEntry(name="imageUrl", source="/document/imageUrl")],  
        outputs=[OutputFieldMappingEntry(name="vector", target_name="imageVector")],  
    )  
  
    skillset = SearchIndexerSkillset(  
        name=skillset_name,  
        description="Skillset to extract image vector",  
        skills=[skill],  
        cors_options=CorsOptions(allowed_origins=["*"]),  
    )  
  
    client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))  
    client.create_skillset(skillset)  
    return skillset  

# Create Indexer doing a field mapping from the output of the Web API skill to the imageVector field  

In [150]:
def sample_indexer_workflow():  
    skillset_name = create_skillset().name  
    print("Skillset is created")  
  
    ds_name = create_data_source().name  
    print("Data source is created")  
  
    ind_name = create_index().name  
    print("Index is created")  
  
    indexer_name = "my-image-indexer"  
    indexer = SearchIndexer(  
        name=indexer_name,  
        data_source_name=ds_name,  
        target_index_name=ind_name,  
        skillset_name=skillset_name,  
        field_mappings=[  
            FieldMapping(source_field_name="metadata_storage_path", target_field_name="id"),  
            FieldMapping(source_field_name="metadata_storage_name", target_field_name="title"),  
        ],  
        output_field_mappings=[OutputFieldMappingEntry(source_field_name="/document/vector", target_field_name="imageVector")],
    )  
  
    indexer_client = SearchIndexerClient(service_endpoint, AzureKeyCredential(key))
    indexer_client.create_indexer(indexer)

    result = indexer_client.get_indexer(indexer_name)  
    print(result)  

    indexer_client.run_indexer(result.name)  

    indexer_status = indexer_client.get_indexer_status(indexer_name)  
    print(indexer_status)
sample_indexer_workflow()

cors_options is not a known attribute of class <class 'azure.search.documents.indexes.models._models.SearchIndexerSkillset'> and will be ignored


Skillset is created


retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored


Data source is created
Index is created


TypeError: __init__() missing 1 required keyword-only argument: 'name'

In [111]:
# Delete a search index
index_client.delete_index(index_name)
print(f'Index {index_name} deleted')


Index python-vector-images-demo deleted
